In [51]:
# import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [52]:
# create a list of all of the urls we will be obtaining pbp data from
urls = []
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2018-11-06-20-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2018-11-10-19-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2018-11-14-18-villanova.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2018-11-17-12-george-washington.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2018-11-18-13-providence.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2018-11-23-16-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2018-11-23-16-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2018-12-01-15-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2018-12-04-21-northwestern.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2018-12-08-12-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2018-12-15-14-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2018-12-22-16-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2018-12-30-12-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-01-03-19-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-01-06-16-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-01-10-20-illinois.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-01-13-19-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-01-19-12-wisconsin.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-01-22-19-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-01-25-18-indiana.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-01-29-21-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-02-01-19-iowa.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-02-05-20-rutgers.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-02-09-12-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-02-12-20-penn-state.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-02-16-12-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-02-21-19-minnesota.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-02-24-15-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-02-28-19-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-03-03-15-maryland.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-03-09-20-michigan-state.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-03-15-21-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-03-16-15-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-03-17-15-michigan-state.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-03-21-21-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-03-23-17-michigan.html')
urls.append('https://www.sports-reference.com/cbb/boxscores/pbp/2019-03-28-21-michigan.html')

In [53]:
# define a function that will create a game log from a BeautifulSoup object passed in as a parameter
def create_gamelog(soup):
    # get all tbody sections
    table = soup.findAll("tbody")

    # locate the tbody section we are looking for by searching for the string "data-stat="time_remaining""
    i = 0
    for entry in table:
        x = str(entry)
        if "data-stat=\"time_remaining\"" in x:
            data = table[i]
        i+= 1

    # specify a list of columns for which we want to extract data from the html
    desired_columns = ['time_remaining', 'visitor_play', 'visitor_pts', 'score', 
                       'home_pts', 'home_play']

    # find all the rows from the play by play chart
    rows = data.find_all('tr')
    
    game_log = []

    # loop through each row in the html
    for row in rows:
        # create a list of the data for each row
        df_row = []
        # make sure there is a header tag in the current row we are looking at
        if (row.find('th', {"scope":"row"}) != None):
            # loop through each of our desired columns
            for c in desired_columns:
                # time_remaining is stored with a header tag
                if c == 'time_remaining':
                    cell = row.find("th",{"data-stat": c})
                # the rest of the columns in the chart are stored with a data tag
                else:
                    cell = row.find("td",{"data-stat": c})
                # make sure the cell actually contains data
                if cell != None:
                    # strip the html and format the data as text/ a string
                    a = cell.text.strip().encode()
                    text=a.decode("utf-8")
                    # add the piece of data to the list for that row
                    df_row.append(text)
        # if the length of the data row is 6 (meaning it is a full row), add it to the list of all rows of data for the chart
        if (len(df_row) == 6):
            game_log.append(df_row)
            
    return game_log  

In [54]:
# define the list of columns for the pandas dataframe
game_data_cols = ['Time Remaining', 'Visitor Play', 'V_PTS', 'Score', 
                  'H_PTS', 'Home Play']

# create a list of all of Michigan's games in the season to be used as key values for a dictionary
# uppercase indicates a home game, lowercase indicates a road game, * indicates a neutral court game
opponents = ['Norfolk State', 'Holy Cross', 'villanova', 'George Washington*', 'Providence*', 'Chattanooga',
            'North Carolina', 'Purdue', 'northwestern', 'South Carolina', 'Western Michigan', 'Air Force',
            'Binghampton', 'Penn State', 'Indiana', 'illinois', 'Northwestern', 'wisconsin', 'Minnesota',
            'indiana', 'Ohio State', 'iowa', 'rutgers', 'Wisconsin', 'penn state', 'Maryland', 'minnesota',
            'Michigan State', 'Nebraska', 'maryland', 'michigan state', 'Iowa*', 'Minnesota*','Michigan State*', 
             'Montana*', 'Florida*', 'Texas Tech*']

# create a dictionary where the keys are opponents and the values are pandas dataframes of the pbp data
pbp_data = {}

i = 0
# loop through each url provided for the 2018-19 season
for url in urls:
    # get the url and read the html using BeautifulSoup
    source = requests.get(url).text
    soup = BeautifulSoup(source)
    # create a gamelog, which is a list of lists of pbp data for an individual game
    g_log = create_gamelog(soup)
    
    # create a pandas dataframe from the gamelog
    game_data = pd.DataFrame(g_log, columns = game_data_cols)
    
    # add the dataframe to the play by play dictionary
    pbp_data[opponents[i]] = game_data
    
    i += 1

In [59]:
for game in pbp_data:
    print(game)
    
pbp_data['maryland'].head(30)

Norfolk State
Holy Cross
villanova
George Washington*
Providence*
Chattanooga
North Carolina
Purdue
northwestern
South Carolina
Western Michigan
Air Force
Binghampton
Penn State
Indiana
illinois
Northwestern
wisconsin
Minnesota
indiana
Ohio State
iowa
rutgers
Wisconsin
penn state
Maryland
minnesota
Michigan State
Nebraska
maryland
michigan state
Iowa*
Minnesota*
Michigan State*
Montana*
Florida*
Texas Tech*


,Time Remaining,Visitor Play,V_PTS,Score,H_PTS,Home Play
0,20:00,Start of game,,0-0,,
1,20:00,Jump ball,,0-0,,
2,20:00,Start of period,,0-0,,
3,19:58,,,0-0,,Bruno Fernando won jump ball
4,19:58,Jon Teske lost jump ball,,0-0,,
5,19:34,,,0-3,+3,Jalen Smith made 3-point field goal
6,19:34,,,0-3,,Darryl Morsell assist
7,19:20,Ignas Brazdeikis missed layup (blocked),,0-3,,
8,19:20,,,0-3,,Bruno Fernando block
9,19:19,Ignas Brazdeikis offensive rebound,,0-3,,
